## khaiii 형태소 복원
* 19/11/27 수정) Jamo클래스 생성함, 인풋이 String타입임을 감안해서 짜여짐. 차후 모델의 아웃풋이 리스트로 바뀔 수 있으므로 주의해야 함. 복원사전은 거의 포기상태
* 19/12/23 수정) 새로운 버전의 모델의 아웃풋에 맞추어 형태소 복원 클래스도 수정되었음
---

* 우리의 딥러닝 모델은 자연스럽게 말하지 못한다.
* ex) '만 18 세 64 세 의 장애 인복지법상 등록 되 ㄴ 지 적 자폐 성 장애인 이 대상 이 ㅂ 니 UNK 이 ㄴ 발달장애 이 ㄴ 중 낮 시간 9 시 18 시 UNK 의 욕구 가 높 은 분 들 중 기존 의 복지 서비스 를 받 지 못하 던 분 들 을 대상 으로 하 ㅂ니다'
* 자연스럽게 만들자는 것이 목표

#### 1 안) 복원사전 이용하기
#### 2 안) Rule-base 복원

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

fatal: destination path 'khaiii' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
-- [khaiii] fused multiply add option enabled
-- [hunter] Calculating Toolchain-SHA1
-- [hunter] Calculating Config-SHA1
-- [hunter] HUNTER_ROOT: /root/.hunter
-- [hunter] [ Hunter-ID: 70287b1 | Toolchain-ID: f1a5aaa | Config-ID: dffbc08 ]
-- [hunter] BOOST_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 1.68.0-p1)
-- Boost version: 1.68.0
-- [hunter] CXXOPTS_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 2.1.1-pre)
-- [hunter] EIGEN_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 3.3.5)
-- [hunter] FMT_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 4.1.0)
-- [hunter] GTEST_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 1.8.0-hunter-p11)
-- [hunter] NLOHMANN_JSON_ROOT: /root/.hunter/_Base/70287b1/f1a5aaa/dffbc08/Install (ver.: 3.3.0)
-- [hunter] SPDLOG_ROOT: /root/.hunte

In [0]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

for word in api.analyze("안녕, 세상."):
    print(word)

안녕,	안녕/IC + ,/SP
세상.	세상/NNG + ./SF


## 1안) 복원사전 만들기
* 복원사전만으로는 힘듦 
* 변칙이 너무 너무 많음 ㅡㅡ


In [0]:
# khaiii 내부에 있는 형태소복원 사전!!!

dic_dir = '/content/khaiii/rsc/src/restore.dic'
with open(dic_dir, 'r') as dic:
    dic = [i for i in dic.readlines()]
dic[:10]

['!/I-EC:I-SF:0\t아/I-EC !/I-SF\n',
 '!/I-EC:I-SF:1\t어/I-EC !/I-SF\n',
 '!/I-EF:I-SF:0\t어/I-EF !/I-SF\n',
 '!/I-EF:I-SF:1\t아/I-EF !/I-SF\n',
 '!/I-EF:I-SF:2\t우/I-EF !/I-SF\n',
 '!/I-EF:I-SF:3\t오/I-EF !/I-SF\n',
 '!/I-SP:I-SF:0\t,/I-SP !/I-SF\n',
 '"/B-SE:I-SS:0\t…/B-SE "/I-SS\n',
 '"/I-EC:I-SS:0\t아/I-EC "/I-SS\n',
 '"/I-EC:I-SS:1\t어/I-EC "/I-SS\n']

In [0]:
import re
# 사전에서 \t 단위로 나누고, I- , \n 삭제
temp_dic = [re.sub('I-|\n','',i).split('\t') for i in dic]
# 인풋 단어에서 ':' 뒷자리 삭제
temp_dic = [[j[:j.index(':')] if ':' in j else j for j in i] for i in temp_dic]
# /와 pos정보 삭제
temp_list = []
for key, value in temp_dic:
    key = key[:key.index('/')] 
    v = value.split(' ')
    value = ' + '.join([i[:i.index('/')] for i in v])
    temp_list.append([key,value])

temp_list[500:510]

[['끔', '끌 + ㅁ'],
 ['끕', '끌 + ㅂ'],
 ['끕', '끄 + ㅂ'],
 ['낀', '끼 + ㄴ'],
 ['낀', '끼 + ㄴ'],
 ['낀', '끼 + ㄴ'],
 ['낀', '끼 + ㄴ'],
 ['낀', '끼 + ㄴ'],
 ['낀', '끼 + ㄴ'],
 ['낀', '끼 + ㄴ']]

In [0]:
final_dic = {j:i for i,j in temp_list}
final_dic

{'아 + !': '!',
 '어 + !': '!',
 '우 + !': '!',
 '오 + !': '!',
 ', + !': '!',
 '… + "': '"',
 '아 + "': '"',
 '어 + "': '"',
 '. + "': '"',
 "… + '": "'",
 "아 + '": "'",
 "어 + '": "'",
 "우 + '": "'",
 "오 + '": "'",
 "ㄹ + '": "'",
 '어 + (': '(',
 '아 + (': '(',
 '아 + )': ')',
 '. + )': ')',
 '아 + ,': ',',
 '어 + ,': ',',
 '우 + ,': ',',
 '오 + ,': ',',
 'ㄹ + ,': ',',
 ', + ,': ',',
 '아 + -': '-',
 '어 + -': '-',
 '아 + .': '.',
 '어 + .': '.',
 '우 + .': '.',
 '오 + .': '.',
 '아 + ': '',
 '어 + ': '',
 '어 + ;': ';',
 '어 + =': '=',
 '아 + =': '=',
 '오 + >': '>',
 '아 + ?': '?',
 '어 + ?': '?',
 '_ + x': 'x',
 '어 + ~': '~',
 '… + .': '…',
 '어 + …': '…',
 '아 + …': '…',
 '우 + …': '…',
 '… + …': '…',
 '아 + ∼': '∼',
 '아 + 《': '《',
 '어 + 가': '가',
 '아 + 가': '가',
 '갑': '가',
 '갛': '가',
 '이 + 가': '가',
 '갈': '가',
 '가 + 아': '가',
 '그 + 아': '가',
 '그 + 어': '궈',
 '가 + ㄴ': '간',
 '아 + 가 + ㄴ': '간',
 '어 + 가 + ㄴ': '간',
 '갛 + ㄴ': '간',
 '갑 + ㄴ': '간',
 '갈 + ㄴ': '간',
 '가 + ㄹ': '갈',
 '아 + 가 + ㄹ': '갈',
 '어 + 가 + ㄹ': '갈',
 '갈 + ㄹ': 

In [0]:
import json

save_dir = "/content/drive/My Drive/ToBigs/chatbot_for_the_disabled/Phase1 : Data Collection & Preprocessing /Data/Tokenizer/Khaiii_restore.json"
with open(save_dir, 'w') as f:
    json.dump(final_dic, f)

## 2안) Rule-base
* 자모조합 클래스와 대충 함수로 복원 가능하게 만들었음


In [0]:
from six import unichr
from khaiii import KhaiiiApi

# Ref: https://github.com/bluedisk/hangul-toolkit/tree/master/hgtk
# 최종목표는 우리의 딥러닝모델이 Khaiii에 기반한 Sequence를 뱉을 때 자연스런 말로 바꿔주기입니다.
# compose : 'ㄴ','ㅏ','ㄴ' -->> '난'
# dempose : '난' -->> 'ㄴ','ㅏ','ㄴ'
# restore : '나','ㄴ' -->> '난'
# checkRestore : 'ㅂ니다' -->> 'ㅂ', '니다'
# token2sentence : Khaiii 토큰 -->> 자연스런 문장 
class Jamo:
    def __init__(self):
        self.CHO = (
            u'ㄱ', u'ㄲ', u'ㄴ', u'ㄷ', u'ㄸ', u'ㄹ', u'ㅁ', u'ㅂ', u'ㅃ', u'ㅅ',
            u'ㅆ', u'ㅇ', u'ㅈ', u'ㅉ', u'ㅊ', u'ㅋ', u'ㅌ', u'ㅍ', u'ㅎ'
        )

        self.JOONG = (
            u'ㅏ', u'ㅐ', u'ㅑ', u'ㅒ', u'ㅓ', u'ㅔ', u'ㅕ', u'ㅖ', u'ㅗ', u'ㅘ',
            u'ㅙ', u'ㅚ', u'ㅛ', u'ㅜ', u'ㅝ', u'ㅞ', u'ㅟ', u'ㅠ', u'ㅡ', u'ㅢ', u'ㅣ'
        )

        self.JONG = (
            u'', u'ㄱ', u'ㄲ', u'ㄳ', u'ㄴ', u'ㄵ', u'ㄶ', u'ㄷ', u'ㄹ', u'ㄺ',
            u'ㄻ', u'ㄼ', u'ㄽ', u'ㄾ', u'ㄿ', u'ㅀ', u'ㅁ', u'ㅂ', u'ㅄ', u'ㅅ',
            u'ㅆ', u'ㅇ', u'ㅈ', u'ㅊ', u'ㅋ', u'ㅌ', u'ㅍ', u'ㅎ'
        )

        self.JAMO = self.CHO + self.JOONG + self.JONG[1:]

        self.NUM_CHO = 19
        self.NUM_JOONG = 21
        self.NUM_JONG = 28

        self.FIRST_HANGUL_UNICODE = 0xAC00  # '가'
        self.LAST_HANGUL_UNICODE = 0xD7A3  # '힣'

    def compose(self, chosung, joongsung, jongsung=u''):
        if jongsung is None: jongsung = u''

        try:
            chosung_index = self.CHO.index(chosung)
            joongsung_index = self.JOONG.index(joongsung)
            jongsung_index = self.JONG.index(jongsung)
        except Exception:
            raise NotHangulException('No valid Hangul character index')

        return unichr(0xAC00 + chosung_index * self.NUM_JOONG *\
                    self.NUM_JONG + joongsung_index * self.NUM_JONG + jongsung_index)

    def decompose_index(self, code):
        jong = int(code % self.NUM_JONG)
        code /= self.NUM_JONG
        joong = int(code % self.NUM_JOONG)
        code /= self.NUM_JOONG
        cho = int(code)
        return cho, joong, jong

    def decompose(self, hangul_letter):
        code = ord(hangul_letter) - self.FIRST_HANGUL_UNICODE
        cho, joong, jong = self.decompose_index(code)

        if cho < 0:
            cho = 0
        try:
            return self.CHO[cho], self.JOONG[joong], self.JONG[jong]
        except:
            print("%d / %d  / %d"%(cho, joong, jong))
            print("%s / %s " %( self.JOONG[joong].encode("utf8"), self.JONG[jong].encode('utf8')))
            raise Exception()

    def restore(self, char1, char2):
        cho, joong,_ = self.decompose(char1)
        return self.compose(cho, joong, char2)

    def checkRestore(self, inp):
    # 만약 맨 앞글자가 자음 하나일 경우, 해당 자음과 나머지 글자 리턴
        if inp[0] in self.CHO:
            return inp[0], inp[1:]
        else:
            return 'Pass', 'Pass'

    # version 1
    # def token2sentence(self, text):
    #     api = KhaiiiApi()
    #     split_input = [str(i).split('\t') for i in api.analyze(text)]
    #     pure_input = [i[0] for i in split_input]
    #     pos_input = [i[1] for i in split_input]
    #     print(pos_input)
    #     new_pure = []

    #     # Restore '하','ㅂ니다' -->> '합','니다'
    #     for i in range(len(split_input)):
    #         inp_0, inp_1 = self.checkRestore(pure_input[i]) 
    #         if inp_0 != 'Pass':
    #             pure_input[i-1] = self.restore(pure_input[i-1], inp_0)
    #             pure_input[i] = inp_1

    # version 2
    def token2sentence(self, token_list):
        pos_input = token_list
        print(pos_input)
        pure_input = [i[:i.index('/')] for i in pos_input]
        print(pure_input)
        new_pure = []

        # Restore '하','ㅂ니다' -->> '합','니다'
        for i in range(len(token_list)):
            inp_0, inp_1 = self.checkRestore(pure_input[i]) 
            if inp_0 != 'Pass':
                pure_input[i-1] = self.restore(pure_input[i-1], inp_0)
                pure_input[i] = inp_1

        # 띄어쓰기 교정
        # J : 조사, E : 어미, XS : 파생 접미사
        for i in range(len(token_list)):
            p = pos_input[i]
            if p[p.index('/')+1] == 'J' or p[p.index('/')+1] == 'E' or p[p.index('/')+1] == 'XS':
                new_pure[-1] = new_pure[-1] + pure_input[i]
            else:
                new_pure.append(pure_input[i])
        new_pure = [i for i in new_pure if len(i) > 0]
        pure_text = ' '.join(new_pure)

        # 우리 도메인에 맞게 '장애 인'을 붙여주자 
        pure_text = pure_text.replace('애 인','애인')

        return pure_text

ModuleNotFoundError: ignored

In [0]:
text = '만 18 세 64 세 의 장애 인복지법상 등록 되 ㄴ 지 적 자폐 성 장애인 이 대상 이 ㅂ 니 UNK 이 ㄴ 발달장애 이 ㄴ 중 낮 시간 9 시 18 시 UNK 의 욕구 가 높 은 분 들 중 기존 의 복지 서비스 를 받 지 못하 던 분 들 을 대상 으로 하 ㅂ니다'
token_list = ['시행법/NNG', '전주장애/NNP', '장애/NNG', '이/VCP', '로/JKB', 'ㄴ/ETM', '부산광역시/NNP', '로/JKB', '는/JX', '수/NNG', '나요/EC', '38/SN', '탈수/NNG', '1/SN', '로/JKB', '로/JKB', '로/JKB', '로/JKB']
jamo = Jamo()
new_text = jamo.token2sentence(token_list)
# print(text)
print(new_text)

NameError: ignored

'나/체언'

* 19/11/27 현재 트랜스포머 모델은 POS정보 없이 학습되어있다.
* POS정보를 학습하면 더 나은 결과를 얻을 수 있을까?